## Industry Comparable Analysis using FinViz Financial Data

Comparable company analysis (or “comps” for short) is a valuation methodology that looks at financials of similar public companies and uses them to derive the value of another business. As a relative form of valuation, this analysis evaluates a company relative to its industry peers using financial ratios such as Price-to-Earnings (P/E) or Price-to-Book (P/B), as well as debt and profitability measures. 

For this notebook, all you have to do is enter the ticker for the company you are interested in and Run All cells. The program will display charts to visualize how the company stacks up to the industry avg while also lining up all comparable companies in later graphs for each comparison. Finally, it will programmatically evaluate the company by determining in how many measurements (out of a total of 14) the company beats the industry average and produces a corresponding rating.

Financial data is pulled from **finviz.com** using **python requests** and parsed using **pd.read_html**.

All graphs are interactive. You can zoom in or hover over for more information.

#### Built With
 * Python 3
 * Plotly

###### Written by: Jake Robinson (https://github.com/jakerobinson19)

In [46]:
ticker = 'MGM'

In [47]:
# import pandas and requests packages
import pandas as pd
import requests

# iPython display packages for hiding underlying code
from IPython.display import HTML
from IPython.display import display

# Plotly visualization packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# global variables (mainly used for standardized url requests)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
url = 'https://finviz.com/quote.ashx?t={}'.format(ticker)
combined_data = []

# METHODS 
# All methods here are available in the comp_analysis_functions.py file in the git repo)
# https://github.com/jakerobinson19/Python-Finance-Notebooks

# Extract the table data from finviz and convert into a pd dataframe
def extract_data_table(ind, code, page, headers=headers):
    
    url = 'https://finviz.com/screener.ashx?v={}&f=ind_{}&r={}1'.format(code,ind,page*2-2)
    html_response = requests.get(url, headers=headers).content

    dfs = pd.read_html(html_response)
    
    pages = dfs[13][3].str.split('/')[0][-1]
    
    data_table = dfs[14]
    headers = data_table.iloc[0]
    data_table = pd.DataFrame(data_table.values[1:], columns=headers).set_index('No.')
    
    return(data_table, int(pages))

# Intake a dataframe of companies to sort into small, mid, or large cap
def assign_cap_groupings(comp_data):
    caps = {'Small Cap': [],
        'Mid Cap': [],
        'Large Cap': []}

    for ticker in comp_data.index:
        mcap = str(comp_data.loc[ticker]['Market Cap'])

        if mcap != 'nan':
            mcap = [int(re.split('(\d+)',mcap)[1]), re.split('(\d+)',mcap)[-1]]
        else:
            continue
            
        if mcap[1] == 'M':
            caps['Small Cap'].append(ticker)
        elif mcap[0] < 10:
            caps['Mid Cap'].append(ticker)
        else:
            caps['Large Cap'].append(ticker)
        
    return(caps)

# Pull a specific tickers data by going to the corresponding finviz page and extracting into pd dataframe
def get_ticker_data(ticker):
    url = 'https://finviz.com/quote.ashx?t={}'.format(ticker)

    response = requests.get(url).content
    ticker_dfs = pd.read_html(response)

    ticker_financials = ticker_dfs[6]

    sector, industry, country = ticker_dfs[5][0][2].split('|')
    ind = format_string(industry)

    return(ticker_financials, ind)

# Using the industry, grab all companies from the finviz industry page and concat into one dataframe
def get_comp_data(ind, headers=headers):
    pg = 1
    
    val_data, pages = extract_data_table(ind, '121', pg)
    fin_data, pages = extract_data_table(ind, '161', pg)
    
    while pg < pages:
        pg += 1
        val_data = pd.concat([val_data, extract_data_table(ind, '121', pg)[0]])
        fin_data = pd.concat([fin_data, extract_data_table(ind, '161', pg)[0]])
    
    comp_data = pd.concat([val_data, fin_data],axis=1)
    comp_data = comp_data.T.drop_duplicates().T
    comp_data.set_index('Ticker',inplace=True)
    comp_data = comp_data.drop(['Earnings','Volume'],axis=1)

    return(comp_data)

# Calculate the industry averages for each financial measurement in the comparable data table
def get_industry_averages(comp_data):
    averages = None

    for column in comp_data:
        try:
            avg = comp_data[column].sum()/len(comp_data)
        except:
            continue
        
        avg = pd.DataFrame([[column,avg]], columns=['Type','Avg'])
    
        if averages is None:
            averages = avg
        else:
            averages = pd.concat([averages, avg])

    return(averages)

# Determine the instances where the ticker in question is better than the industry average
# Store each measurement where the ticker "beats" its competitors and tally up the total
def calculate_beats(tick_data, averages):
    beat_number = 0
    finrat_beats = []
    dbt_beats = []
    marg_beats = []
    
    for ind, value in enumerate(tick_data): 
        #print(ind,value,averages.iloc[ind]['Type'], float(averages.iloc[ind]['Avg']))
        if ind >= 0 and ind < 7:
            if value != 0 and value < float(averages.iloc[ind]['Avg']):
                beat_number += 1
                finrat_beats.append(averages.iloc[ind]['Type'])
        if ind == 18 or ind == 19:
            if value != 0 and value > float(averages.iloc[ind]['Avg']):
                beat_number += 1
                dbt_beats.append(averages.iloc[ind]['Type'])
        if ind == 20 or ind == 21:
            if value < float(averages.iloc[ind]['Avg']):
                beat_number += 1
                dbt_beats.append(averages.iloc[ind]['Type'])
        if ind > 21:
            if value > float(averages.iloc[ind]['Avg']):
                beat_number += 1
                marg_beats.append(averages.iloc[ind]['Type'])
            
    if beat_number < 6:
        sentiment = ('Poorly', C)
    elif beat_number < 10: 
        sentiment = ('Fairly', 'B')
    else: 
        sentiment = ('Favorably', 'A')
        
    return(beat_number, sentiment, finrat_beats, dbt_beats, marg_beats)

# Simple formatting function to change all table values to floats for calculation purposes
def format_data_to_floats(table):
    for col in table:
        table[col] = table[col].str.strip("%").replace('-',0)
        table[col] = pd.to_numeric(table[col], errors='ignore')
        
# Formatting for urls        
def format(data_string):
    return(data_string.lower().replace(" ","").replace("-","").replace("&",""))        

# Fast method for assigning a color to the plotly graphs
def get_chart_color(tick_index, graph_num):
    color_options = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd','#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    colors = [color_options[graph_num],]*len(comp_data.index)
    colors[tick_index] = 'gold'
    
    return(colors)

# method used exclusively for hiding the code in the jupyter notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')

# Initial calls to grab and parse the finviz data for the designated ticker
response = requests.get(url, headers=headers).content
ticker_dfs = pd.read_html(response)

ticker_financials = ticker_dfs[6]

sector, industry, country = ticker_dfs[5][0][2].split('|')
ind = format(industry)

display(tag)

## Financials

In [48]:
ticker_financials

,0,1,2,3,4,5,6,7,8,9,10,11
0,Index,S&P 500,P/E,3.97,EPS (ttm),5.56,Insider Own,0.60%,Shs Outstand,495.42M,Perf Week,21.21%
1,Market Cap,10.88B,Forward P/E,-,EPS next Y,-0.92,Insider Trans,138.69%,Shs Float,489.44M,Perf Month,41.50%
2,Income,2.83B,PEG,-,EPS next Q,-1.72,Inst Own,77.50%,Short Float,6.53%,Perf Quarter,6.11%
3,Sales,11.98B,P/S,0.91,EPS this Y,378.50%,Inst Trans,-11.85%,Short Ratio,0.99,Perf Half Y,-31.43%
4,Book/sh,16.50,P/B,1.34,EPS next Y,47.10%,ROA,8.40%,Target Price,18.04,Perf Year,-19.55%
5,Cash/sh,12.20,P/C,1.81,EPS next 5Y,-,ROE,39.70%,52W Range,5.90 - 34.63,Perf YTD,-33.69%
6,Dividend,0.01,P/FCF,130.46,EPS past 5Y,71.20%,ROI,17.40%,52W High,-36.31%,Beta,2.23
7,Dividend %,0.05%,Quick Ratio,2.90,Sales past 5Y,5.10%,Gross Margin,40.10%,52W Low,273.89%,ATR,1.51
8,Employees,52000,Current Ratio,2.90,Sales Q/Q,-29.10%,Oper. Margin,40.30%,RSI (14),67.91,Volatility,9.00% 7.55%
9,Optionable,Yes,Debt/Eq,1.47,EPS Q/Q,-,Profit Margin,23.60%,Rel Volume,0.97,Prev Close,23.76


## Comparable Industry Financials

In [49]:
comp_data = get_comp_data(ind)
comp_data

,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,EPS this Y,EPS next Y,...,ROA,ROE,ROI,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M
Ticker,,,,,,,,,,,,,,,,,,,,,
BBX,178.25M,-,76.67,-,0.21,0.40,0.45,3.59,-47.30%,104.80%,...,-0.70%,-2.20%,-4.60%,-,-,1.58,1.58,63.70%,-10.40%,-1.50%
BXG,353.29M,20.94,10.85,-,0.48,1.05,1.46,-,-59.80%,212.12%,...,1.50%,4.90%,4.40%,-,-,1.81,1.81,76.80%,5.00%,2.70%
BYD,2.51B,-,72.23,-,0.79,2.42,3.02,11.84,38.80%,107.30%,...,-0.50%,-2.90%,7.80%,1.90,1.90,3.93,3.95,47.20%,5.70%,-1.10%
CNTY,160.84M,-,-,-,0.62,1.58,2.52,-,-674.10%,97.30%,...,-12.70%,-42.10%,-1.50%,1.50,1.40,4.40,4.50,46.60%,-15.60%,-25.40%
CZR,8.29B,-,-,-,0.98,3.67,3.10,-,-772.70%,42.70%,...,-3.10%,-33.20%,3.70%,1.20,1.20,8.37,8.77,51.90%,3.70%,-9.30%
ERI,3.25B,-,-,-,1.37,3.53,4.62,26.62,-15.30%,92.30%,...,-2.20%,-12.40%,7.70%,1.60,1.60,3.99,4.25,49.70%,6.60%,-5.60%
FLL,54.44M,-,-,-,0.35,1.06,2.24,-,-28.40%,100.00%,...,-4.10%,-15.20%,3.70%,1.40,1.30,2.06,2.09,43.10%,1.00%,-5.50%
GDEN,367.88M,-,-,-,0.39,1.47,1.22,-,-92.40%,65.50%,...,-3.70%,-22.40%,2.70%,2.60,2.60,5.14,5.17,39.50%,1.20%,-6.80%
HGV,2.07B,12.77,12.68,0.90,1.19,3.79,3.09,13.80,-19.10%,930.53%,...,5.30%,32.50%,12.00%,-,-,3.82,3.82,23.20%,14.00%,9.70%


In [50]:
# Small cap: Under $1 Billion
# Mid Cap: $2 to $10 Billion
# Large Cap: Over $10 Billion

# Format all values in the above table to float for calculations and assign cap groupings
format_data_to_floats(comp_data)
caps = assign_cap_groupings(comp_data)

# Find the cap grouping of our ticker
for index, c in enumerate(caps.values()):
    if ticker in c:
        tick_cap = list(caps.keys())[index]

display(tag)

# Print only the companies in the same cap group as our ticker
# caps[tick_cap] will return the list of comparable tickers
print("{} Comparables for {}".format(tick_cap, ticker))
comp_data.loc[caps[tick_cap]]

Large Cap Comparables for MGM


,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,EPS this Y,EPS next Y,...,ROA,ROE,ROI,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M
Ticker,,,,,,,,,,,,,,,,,,,,,
LVS,40.50B,19.53,21.41,0.0,3.41,9.13,15.57,0.00,11.4,6375.0,...,9.5,41.1,21.3,1.4,1.4,2.73,2.74,49.3,27.9,17.8
MGM,10.88B,3.97,0.00,0.0,0.91,1.34,1.81,130.46,378.5,47.1,...,8.4,39.7,17.4,2.9,2.9,1.46,1.47,40.1,40.3,23.6
WYNN,10.40B,0.00,203.89,0.0,1.76,8.83,0.00,0.00,-32.2,104.5,...,-2.8,-22.1,5.7,1.8,1.8,8.96,9.15,35.8,6.1,-6.5


## Compare Security Against Industry Averages

In [51]:
ratio_start, ratio_end = 1, 7
debt_start, debt_end = 18,22
margin_start, margin_end = 22,26

# generate list of financials for ticker in question
tick_data = list(comp_data.loc[ticker])
del tick_data[0]

# calculate the industry averages from the comparable data and store in a df
averages = get_industry_averages(comp_data)

# perform analysis to compare the ticker against the industry peers for later discussion
beat_number, sentiment, finrat_beats, dbt_beats, marg_beats = calculate_beats(tick_data, averages)

display(tag)

tick_index = list(comp_data.index).index(ticker)  

# all code below this creates and fancies up the plotly graphs
fig = go.Figure()

fig = make_subplots(rows=3, cols=1, start_cell="top-left", subplot_titles=("Financial Ratios", "Debt", "Margins"))


fig.add_trace(go.Bar(x=averages['Type'][:ratio_end],
                y=tick_data[:ratio_end],
                name=ticker,
                marker_color='#1f77b4'),
              row=1, col=1)

fig.add_trace(go.Bar(x=averages['Type'][:ratio_end],
                y=averages['Avg'][:ratio_end],
                name='Industry Avg',
                marker_color='#ff7f0e'), 
              row=1, col=1)

fig.add_trace(go.Bar(name=ticker, x=averages['Type'][debt_start:debt_end],
                     y=tick_data[debt_start:debt_end], 
                     textposition='auto',
                     marker_color='#1f77b4'),
              row=2, col=1)

fig.add_trace(go.Bar(name="Ind Avg", x=averages['Type'][debt_start:debt_end],
                     y=averages['Avg'][debt_start:debt_end], 
                     textposition='auto',
                     marker_color='#9467bd'),
              row=2, col=1)

fig.add_trace(go.Bar(name=ticker, x=averages['Type'][margin_start:margin_end],
                     y=tick_data[margin_start:margin_end], 
                     textposition='auto',
                     marker_color='#1f77b4'),
              row=3, col=1)

fig.add_trace(go.Bar(name="Ind Avg", x=averages['Type'][margin_start:margin_end],
                     y=averages['Avg'][margin_start:margin_end], 
                     textposition='auto',
                     marker_color='#bcbd22'),
              row=3, col=1)

fig.update_layout(
    title='{} vs Industry Avgs'.format(ticker),
    xaxis_tickfont_size=14,
    yaxis=dict(
        titlefont_size=16,
        tickfont_size=14,
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    showlegend=False)

fig.show()

# Comp Analysis

Drill down further to see how the security stacks up directly to each of its industry peers

## Financial Ratios

Low ratios comparable to competitors can be an indicator of value. The lower the ratio, the cheaper the stock is. Unless of course, the value is zero, which is often because the company is not generating positive earnings or cash flow so you are unable to calculate the ratio.

In [52]:
display(tag)

fig = make_subplots(rows=3, cols=2, start_cell="top-left", subplot_titles=("P/E", "Fwd P/E","P/S", "P/B", "P/C","P/FCF"))

fig.add_trace(go.Bar(name="P/E", x=comp_data.index, y=comp_data['P/E'], text=comp_data['P/E'], textposition='outside', marker_color=get_chart_color(tick_index,0)), row=1, col=1)
fig.add_trace(go.Bar(name="Fwd P/E", x=comp_data.index, y=comp_data['Fwd P/E'], text=comp_data['Fwd P/E'], textposition='auto', marker_color=get_chart_color(tick_index,1)), row=1, col=2)
fig.add_trace(go.Bar(name="P/S", x=comp_data.index, y=comp_data['P/S'], text=comp_data['P/S'], textposition='auto', marker_color=get_chart_color(tick_index,2)), row=2, col=1)
fig.add_trace(go.Bar(name="P/B", x=comp_data.index, y=comp_data['P/B'], text=comp_data['P/B'], textposition='auto', marker_color=get_chart_color(tick_index,3)), row=2, col=2)
fig.add_trace(go.Bar(name="P/C", x=comp_data.index, y=comp_data['P/C'], text=comp_data['P/C'], textposition='auto', marker_color=get_chart_color(tick_index,4)), row=3, col=1)
fig.add_trace(go.Bar(name="P/FCF", x=comp_data.index, y=comp_data['P/FCF'], text=comp_data['P/FCF'], textposition='auto', marker_color=get_chart_color(tick_index,5)), row=3, col=2)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=18)

fig.update_layout(
    autosize=False,
    width=1000,
    height=1200,
    showlegend=False)

fig.show()

## Debt

Debt ratios such as the Current Ratio and Quick ratio are preferred to be higher because its an indication that the company has enough liquidity to be able to pay off and handle its debt. 

Conversely, lower Debt/Equity and LTDebt/Equity values are better because a company that has less debt comparable to its assets holds less risk of not being able to pay those debts and going bankrupt.

In [53]:
display(tag)

fig = make_subplots(rows=2, cols=2, start_cell="top-left", subplot_titles=("Current Ratio", "Quick Ratio","Debt/Equity", "LT Debt/Equity",))

fig.add_trace(go.Bar(name="Curr R", x=comp_data.index, y=comp_data['Curr R'], text=comp_data['Curr R'], textposition='outside', marker_color=get_chart_color(tick_index,6)), row=1, col=1)
fig.add_trace(go.Bar(name="Quick R", x=comp_data.index, y=comp_data['Quick R'], text=comp_data['Quick R'], textposition='auto', marker_color=get_chart_color(tick_index,7)), row=1, col=2)
fig.add_trace(go.Bar(name="Debt/Eq", x=comp_data.index, y=comp_data['Debt/Eq'], text=comp_data['Debt/Eq'], textposition='auto', marker_color=get_chart_color(tick_index,8)), row=2, col=1)
fig.add_trace(go.Bar(name="LTDebt/Eq", x=comp_data.index, y=comp_data['LTDebt/Eq'], text=comp_data['LTDebt/Eq'], textposition='auto', marker_color=get_chart_color(tick_index,9)), row=2, col=2)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8)

fig.update_layout(
    autosize=True,
    width=1000,
    height=1000,
    showlegend=False)

fig.show()

## Margins and Dividend

As far as margins go, it is better to see higher margins and dividends because it demonstrates that the company is able to cover costs more effectively and generate profits, which ultimately benefits the investor.

In [54]:
display(tag)

fig = make_subplots(rows=2, cols=2, start_cell="top-left", subplot_titles=("Gross Margin", "Operating","Profit", "Dividend"))

fig.add_trace(go.Bar(name="Gross Margin", x=comp_data.index, y=comp_data['Gross M'], text=comp_data['Gross M'], textposition='outside', marker_color=get_chart_color(tick_index,1)), row=1, col=1)
fig.add_trace(go.Bar(name="Operating Margin", x=comp_data.index, y=comp_data['Oper M'], text=comp_data['Oper M'], textposition='auto', marker_color=get_chart_color(tick_index,2)), row=1, col=2)
fig.add_trace(go.Bar(name="Profit Margin", x=comp_data.index, y=comp_data['Profit M'], text=comp_data['Profit M'], textposition='auto', marker_color=get_chart_color(tick_index,3)), row=2, col=1)
fig.add_trace(go.Bar(name="Dividend", x=comp_data.index, y=comp_data['Dividend'], text=comp_data['Dividend'], textposition='auto', marker_color=get_chart_color(tick_index,4)), row=2, col=2)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8)

fig.update_layout(
    autosize=True,
    width=1000,
    height=1000,
    showlegend=False)

fig.show()

# Summary 

In [55]:
from random import randint

display(tag)

averages = averages.set_index(averages['Type'])
averages = averages.drop(['Type'], axis=1)

def print_summary(beat_list, type):
    adj_options = ["great", "promising", "preferable", "excellent", "favorable", "superior","better","more desirable"]
    
    print("{} won {} time(s) when it came to {}".format(ticker, len(beat_list), type))
    
    if len(beat_list) > 0:
        for i, item in enumerate(beat_list):
                print("\t{} \033[1m{}\033[0m: it has a {} {} of \033[1m{}\033[0m compared to an industry average of \033[1m{}\033[0m".format("\033[1m*\033[0m", item, adj_options[randint(0, len(adj_options)-1)], item, comp_data.loc[ticker][item], str(round(averages.loc[item][0], 2))))
    else:
        print("{} was not better than the industry average for any of these measures".format(ticker))


print("Overall, {} performs {} against its industrial peers as it beat them in {} out of 15 measurements.".format(ticker, sentiment[0].lower(), beat_number))

print("\nFINANCIALS\n")
print_summary(finrat_beats, "fundamental financial ratios")

print("\nDEBT\n")
print_summary(dbt_beats , "debt ratios")

print("\nMARGINS\n")
print_summary(marg_beats, "margins")


Overall, MGM performs favorably against its industrial peers as it beat them in 10 out of 15 measurements.

FINANCIALS

MGM won 4 time(s) when it came to fundamental financial ratios
	* P/E: it has a better P/E of 3.97 compared to an industry average of 91.71
	* P/S: it has a more desirable P/S of 0.91 compared to an industry average of 1.32
	* P/B: it has a excellent P/B of 1.34 compared to an industry average of 3.05
	* P/C: it has a great P/C of 1.81 compared to an industry average of 4.91

DEBT

MGM won 4 time(s) when it came to debt ratios
	* Curr R: it has a superior Curr R of 2.9 compared to an industry average of 1.69
	* Quick R: it has a great Quick R of 2.9 compared to an industry average of 1.65
	* LTDebt/Eq: it has a superior LTDebt/Eq of 1.46 compared to an industry average of 3.81
	* Debt/Eq: it has a excellent Debt/Eq of 1.47 compared to an industry average of 3.81

MARGINS

MGM won 2 time(s) when it came to margins
	* Oper M: it has a great Oper M of 40.3 compared to an

# FINAL RATING

In [57]:
print(sentiment[1])

A
